# Tanzania Tourism Prediction - Prognozy dotyczące turystyki w Tanzanii
<h2>Autorzy:</h2><br>
<ul>
<li>Piotr Janiszek 247678</li>
<li>Kacper Białek 247629</li>
<li>Franciszek Pawlak 247756</li>
<li>Michał Korblit 242427</li>
</ul>

<h3>Imports</h3>

In [ ]:
import math

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

<h3>Data loadind</h3>


In [ ]:
df_train = pd.read_csv('data/Train.csv')
df_test = pd.read_csv('data/Test.csv')

print(df_train.columns)

<h3>Data Cleaning</h3>

In [ ]:
count = df_train.isna().any(axis=1).sum()
df_train.loc[df_train['most_impressing'].isna(), 'most_impressing'] = 'No comments'
count2 = df_train.isna().any(axis=1).sum()

mask_valid = (df_train['total_male'].notna()) & \
             (df_train['total_female'].notna()) & \
             ((df_train['total_male'] + df_train['total_female']) != 0)
df_train.loc[mask_valid, 'total_cost_per_person'] = df_train.loc[mask_valid, 'total_cost'] / (
            df_train.loc[mask_valid, 'total_male'] + df_train.loc[mask_valid, 'total_female'])
mean = df_train.loc[mask_valid, 'total_cost_per_person'].mean()
print("Średnia bez wierszy z zerową sumą:", mean)


# idzie przez kolumny z nan
for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    if (pd.isna(row['total_male']) | pd.isna(row['total_female'])) & (not (pd.isna(row['total_male']) & pd.isna(row['total_female']))):
        if pd.isna(row['total_male']):
            difference = row['total_cost'] - (row['total_female'] * mean)
            person_left = difference / mean
            person_left = round(person_left, 0)

            if person_left < 0:
                person_left = 0
            #print(f"Inserting {person_left}")
            df_train.loc[index, 'total_male'] = person_left

        else:
            difference = row['total_cost'] - (row['total_male'] * mean)

            person_left = difference / mean
            person_left = round(person_left, 0)

            if person_left < 0:
                person_left = 0
            #print(f"Inserting {person_left}")
            df_train.loc[index, 'total_female'] = person_left

for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    # Completing the travel_with field with the value Alone, when the number of people shows that he travels alone
    if pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) == 1):
        df_train.loc[index, 'travel_with'] = 'Alone'


In [ ]:
# Random forest imputation
mask_valid = (df_train['total_male'].notna()) & \
             (df_train['total_female'].notna()) & \
             ((df_train['total_male'] + df_train['total_female']) != 0)
df_train.loc[mask_valid, 'total_cost_per_person'] = df_train.loc[mask_valid, 'total_cost'] / (
            df_train.loc[mask_valid, 'total_male'] + df_train.loc[mask_valid, 'total_female'])
mean = df_train.loc[mask_valid, 'total_cost_per_person'].mean()

features = ['country', 'age_group', 'total_female',
       'total_male', 'purpose', 'main_activity', 'info_source',
       'tour_arrangement', 'package_transport_int', 'package_accomodation',
       'package_food', 'package_transport_tz', 'package_sightseeing',
       'package_guided_tour', 'package_insurance', 'night_mainland',
       'night_zanzibar', 'payment_mode', 'first_trip_tz', 'most_impressing',
       'total_cost']

features_cat = ['country', 'age_group', 'purpose', 'main_activity', 'info_source', 'tour_arrangement',
                'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz',
                'package_sightseeing', 'package_guided_tour', 'package_insurance',
                'payment_mode', 'first_trip_tz', 'most_impressing']

# Określ dozwolone kategorie - Model 1
allowed_categories = ['Friends/Relatives', 'Children']
# Filtrowanie danych treningowych - uwzględniamy tylko dozwolone kategorie
df_train_imp = df_train[df_train['travel_with'].isin(allowed_categories)].copy()
X_train = pd.get_dummies(df_train_imp[features], columns=features_cat)
y_train = df_train_imp['travel_with']
# Trenowanie modelu
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Określ dozwolone kategorie - Model 2
allowed_categories2 = ['Children', 'Friends/Relatives', 'Spouse', 'Spouse and Children']
# Filtrowanie danych treningowych - uwzględniamy tylko dozwolone kategorie
df_train_imp2 = df_train[df_train['travel_with'].isin(allowed_categories)].copy()
X_train2 = pd.get_dummies(df_train_imp2[features], columns=features_cat)
y_train2 = df_train_imp2['travel_with']
# Trenowanie modelu
rf2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf2.fit(X_train2, y_train2)

for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    # Completing the travel_with field with the estimated value when only one gender participated in the trip and the number of people exceeds one
    if (pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) > 1) &
            ((row['total_male'] == 0 | pd.isna(row['total_male'])) | (row['total_female'] == 0 | pd.isna(row['total_female'])))):

        dummy_df = pd.get_dummies(row[features], columns=features_cat)
        dummy_df.columns = dummy_df.columns.astype(str)
        dummy_df = dummy_df.reindex(columns=X_train.columns, fill_value=0)

        predicted_value = rf.predict(dummy_df)

        #print(predicted_value[0])
        df_train.loc[index, 'travel_with'] = predicted_value[0]

    elif (row['total_male'] + row['total_female'] > 1) & (pd.isna(row['travel_with'])):

        dummy_df = pd.get_dummies(row[features], columns=features_cat)
        dummy_df.columns = dummy_df.columns.astype(str)
        dummy_df = dummy_df.reindex(columns=X_train2.columns, fill_value=0)

        predicted_value = rf2.predict(dummy_df)

        print(predicted_value[0])
        df_train.loc[index, 'travel_with'] = predicted_value[0]

    elif (not pd.isna(row['travel_with'])) & (row['total_male'] == 0 | pd.isna(row['total_male'])) & (row['total_female'] == 0 | pd.isna(row['total_female'])):
        print("Brakuje liczby osob")


        if row['travel_with'] == 'Alone':
            print('predykcja płci')

    elif pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) == 0):
        persons = round(row['total_cost'] / mean, 0)

        if persons <= 1:
            df_train.loc[index, 'travel_with'] = 'Alone'
        else:
            print('predykcja płci i rozłożenia')


    elif pd.isna(row['travel_with']):
        print(f"{row['travel_with']}"
              f"\n{row['total_male']}"
              f"\n{row['total_female']}")


mask_valid = (df_train['total_male'].notna()) & \
             (df_train['total_female'].notna()) & \
             ((df_train['total_male'] + df_train['total_female']) != 0)
df_train.loc[mask_valid, 'total_cost_per_person'] = df_train.loc[mask_valid, 'total_cost'] / (
            df_train.loc[mask_valid, 'total_male'] + df_train.loc[mask_valid, 'total_female'])

count3 = df_train.isna().any(axis=1).sum()

print("Początkowa liczba wierszy z brakującymi wartościami:", count)
print("Liczba wierszy z brakującymi wartościami po uzupełnieniu braków w kolumnie most_impressing:", count2)
print("Końcowa liczba wierszy z brakującymi wartościami:", count3)

for column in df_train.columns:
    empty_count = df_train[column].isna().sum()

    if empty_count > 0:
        print(f"Column '{column}' has {empty_count} empty fields (NaN).")

for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    #print(row)
    pass

df_train.drop(["most_impressing"], axis=1, inplace=True)
